# **Data Witches**

| **Name**         | **Student ID** |
|------------------|----------------|
| Claessen, VVHJAE | i6339543       |
| Ovsiannikova, AM | i6365923       |
| Pubben, J        | i6276134       |
| Roca Cugat, M    | i6351071       |
| Záboj, J         | i6337952       |

# **Logbook**

## Methods

Let's ensure we all use the same names for all components.

| **Variable**                 | **Name**                                      |
|------------------------------|-----------------------------------------------|
| Raw ECG dataframe            | df                                            |
| Label dataframe              | df_labels                                     |
| HRV features (train)         | hrv_train                                     |
| HRV features (test)          | hrv_test                                      |
| HRV extraction type          | FULL (nk.hrv — time + freq + nonlinear + RSA) |
| Clean HRV dataframe (train)  | hrv_train_clean                               |
| Clean HRV dataframe (test)   | hrv_test_clean                                |
| HRV + labels (train)         | hrv_train_with_labels                         |
| Winsorized HRV column        | HRV_MedianNN_winsor                           |
| Model feature matrix (train) | X_train                                       |
| Model feature matrix (test)  | X_test                                        |
| Model target vector (train)  | y_train                                       |
| Model target vector (test)   | y_test                                        |


| **Function**              | **Description**                                | **Arguments**                                |
|---------------------------|------------------------------------------------|----------------------------------------------|
| corr_plot_hrv()           | Correlation plot for HRV features              | df, cols=None                                |
| distplots_hrv()           | Distribution plots (hist + KDE)                | df, cols=None                                |
| boxplots_hrv()            | Boxplots for selected HRV variables            | df, cols                                     |
| check_missing_hrv()       | Missingness summary                            | df                                           |
| identify_outliers()       | IQR-based outlier detection                    | df, column_name, threshold=1.5               |
| model_evaluation()        | Confusion matrix + classification report       | model                                        |
| model_desc()              | Accuracy, CV, ROC-AUC, model performance       | model                                        |


# Preface

## Packages imports

In [ ]:
try:
    print("Loading required packages...")
    import sys
    import random
    import os.path
    import warnings
    from tqdm import tqdm

    import numpy as np
    import pandas as pd
    import seaborn as sns
    import neurokit2 as nk
    from scipy import stats
    import scipy.signal as signal
    from scipy.signal import welch
    import matplotlib.pyplot as plt
    from joblib.testing import xfail
    import plotly.graph_objects as go
    from colorama import Fore, Back, Style
    from plotly.subplots import make_subplots

    from sklearn import tree
    from sklearn.pipeline import Pipeline
    from sklearn.compose import make_column_transformer
    from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

    from sklearn.neural_network import MLPClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
    from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier

    from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, \
        f1_score, precision_score, recall_score, roc_auc_score, roc_curve, RocCurveDisplay, precision_recall_curve
    from sympy import false

    print("Loading successful!")
except Exception:
    print("Installing required packages...")
    !pip install -r https://raw.githubusercontent.com/MAI3003-Data-Witches/AtrialFibrillation-detection/refs/heads/challenge/requirements.txt

    print("Loading required packages...")
    import sys
    import random
    import os.path
    import warnings
    from tqdm import tqdm

    import numpy as np
    import pandas as pd
    import seaborn as sns
    import neurokit2 as nk
    from scipy import stats
    import scipy.signal as signal
    from scipy.signal import welch
    import matplotlib.pyplot as plt
    from joblib.testing import xfail
    import plotly.graph_objects as go
    from colorama import Fore, Back, Style
    from plotly.subplots import make_subplots

    from sklearn import tree
    from sklearn.pipeline import Pipeline
    from sklearn.compose import make_column_transformer
    from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

    from sklearn.neural_network import MLPClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
    from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier

    from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, \
        f1_score, precision_score, recall_score, roc_auc_score, roc_curve, RocCurveDisplay, precision_recall_curve
    from sympy import false

    print("Loading successful!")

## Options settings

In [ ]:
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
random.seed(3003)
IN_COLAB = 'google.colab' in sys.modules
DATA_PRESENT = os.path.isfile("data/Physionet2017Training.tar.xz")
LoadPremadeDataset = True

## Dataset download

In [ ]:
dataset_location = 'data/Physionet2017TrainingData.csv'

In [ ]:
if not DATA_PRESENT:
    !mkdir data
    !wget https://github.com/MAI3003-Data-Witches/Data-Witches_Project2/raw/refs/heads/main/data/Physionet2017Training.tar.xz -O data/Physionet2017Training.tar.xz
    !tar -xf data/Physionet2017Training.tar.xz -C data
else:
    print(f"You already have the dataset downloaded at {dataset_location}, skipping")

In [ ]:
df = pd.read_csv(dataset_location, header=None, index_col=False) * 1000  # Load the dataset already in mV

df.head()

# Data preprocessing
## Extract ECG signals and class labels

In [ ]:
df_labels = pd.read_csv('data/Physionet2017TrainingLabels.csv', header=None, names=['label'])
df_labels['classification'] = df_labels['label'].replace({"N": 0, "A": 1})
df_labels['label'] = df_labels['label'].replace({"N": 'Normal Sinus Rhythm', "A": 'Atrial Fibrillation'})

In [ ]:
df_labels

## Dataset splitting

In [ ]:
df_labeled = pd.merge(df_labels.drop(columns='label'), df, left_on='classification', right_index=True)

train_idx, test_idx = train_test_split(
    df.index,
    test_size=0.2,
    stratify=df_labels["label"],
    random_state=3003
)

print("Train size:", len(train_idx))
print("Test size:", len(test_idx))

# Exploratory Data Analysis
## Dataset characteristics

In [ ]:
num_ecgs = len(df)  # Number of ECGs

num_samples = df.shape[1]  # Number of samples per ECG

sampling_frequency = 300  #Hz
duration = num_samples / sampling_frequency  # Duration of each ECG

class_distribution = df_labels['label'].value_counts()  # Distribution over classes

print(f"Number of ECGs: {num_ecgs}")
print(f"Number of samples per ECG: {num_samples}")
print(f"Duration of each ECG: {duration} seconds")
print(f"\nClass Distribution:\n{class_distribution}")

In [ ]:
# Indices per class (based on df_labels)
sinus_indices = df_labels[df_labels["label"] == "Normal Sinus Rhythm"].index.tolist()
af_indices = df_labels[df_labels["label"] == "Atrial Fibrillation"].index.tolist()

example_sinus_idx = random.choice(sinus_indices)
example_af_idx = random.choice(af_indices)

ecg_sinus_raw = df.iloc[example_sinus_idx].astype(float).values
ecg_af_raw = df.iloc[example_af_idx].astype(float).values

time = np.arange(0, len(ecg_sinus_raw)) / sampling_frequency

In [ ]:
# Summary statistics for each ECG
summary_stats = df.describe().T
summary_stats = pd.concat([summary_stats, df_labels], axis=1)

# Feature extraction

## ECG feature engineering

In [ ]:
# Select an ECG in Normal Sinus Rhythm and one in AF and process them
selected_sinus_indices = random.sample(sinus_indices, 1)
selected_af_indices = random.sample(af_indices, 1)

ecg_NSR = df.iloc[selected_sinus_indices[0]].astype(float)
signals_NSR, info_NSR = nk.ecg_process(ecg_NSR, sampling_rate=sampling_frequency)

plt.rcParams['figure.figsize'] = [12, 4]
nk.ecg_plot(signals_NSR, info_NSR)

ecg_AF = df.iloc[selected_af_indices[0]].astype(float)
signals_AF, info_AF = nk.ecg_process(ecg_AF, sampling_rate=sampling_frequency)

nk.ecg_plot(signals_AF, info_AF)

#### R-peaks**

In [ ]:
# Find R-peaks
peaks_NSR, info_NSR = nk.ecg_peaks(ecg_NSR, sampling_rate=sampling_frequency, correct_artifacts=True, show=True)
peaks_AF, info_AF = nk.ecg_peaks(ecg_AF, sampling_rate=sampling_frequency, correct_artifacts=True, show=True)

#### Time-domain features

In [ ]:
# Time domain features NSR
hrv_time_NSR = nk.hrv_time(peaks_NSR, sampling_rate=sampling_frequency, show=True)
hrv_time_NSR

In [ ]:
# Time domain features AF
hrv_time_AF = nk.hrv_time(peaks_AF, sampling_rate=sampling_frequency, show=True)
hrv_time_AF

### FULL HRV feature extraction for all ECGs (TRAIN)

In [ ]:
#Getting all the ECG readouts so we can extract P-wave information

def get_ECG_readout():
    test_run = 0
    ecg_full = pd.DataFrame()  # Initialize an empty DataFrame

    for i in tqdm(train_idx):

        ecg = df.iloc[i].astype(float)
        signals, info = nk.ecg_process(ecg, sampling_rate=sampling_frequency)

        # Assign the current ecg_index to the signals DataFrame before concatenation
        signals["ecg_index"] = i

        ecg_full = pd.concat([ecg_full, signals], ignore_index=True)

        #test_run += 1

        if test_run == 10:
            break  # Stop after 10 iterations for the example

    return ecg_full

In [ ]:
if LoadPremadeDataset == False:
    ecg_full = get_ECG_readout()

In [ ]:
def get_ECG_metrics(ecg_full):
    ecg_metrics_list = []

    for i in tqdm(train_idx[:]):
        mean_quality = ecg_full.loc[ecg_full.ecg_index == i]['ECG_Quality'].mean()
        mean_pwave_amplitude = ecg_full.loc[(ecg_full.ecg_index == i) & (ecg_full['ECG_P_Peaks'] == 1)][
            'ECG_Clean'].mean()  #You could consider taking sqrt, mean and then **2
        #(more robust) to outliers
        stdev_pwave = ecg_full.loc[(ecg_full.ecg_index == i) & (ecg_full['ECG_P_Peaks'] == 1)]['ECG_Quality'].std()
        #Perhaps I could add something about irregularly irregular rhythm, but it's (really) difficult mathematically
        ecg_metrics_list.append({
            'Mean_Quality': mean_quality,
            'Mean_PWave_Amplitude': mean_pwave_amplitude,
            'STDEV_Pwave': stdev_pwave,
            'ecg_index': i
        })

    return pd.DataFrame(ecg_metrics_list)

In [ ]:
if LoadPremadeDataset == False:
    ecg_metrics = get_ECG_metrics(ecg_full)

In [ ]:
if LoadPremadeDataset == False:
    #FULL HRV feature extraction for all ECGs (TRAIN)

    hrv_features_train = []

    for i in tqdm(train_idx, desc="HRV (ALL FEATURES): TRAIN SET"):
        # Grab raw ECG
        ecg = df.iloc[i].astype(float).values

        try:
            # 1. Clean ECG
            ecg_clean = nk.ecg_clean(ecg, sampling_rate=sampling_frequency)

            # 2. Detect R-peaks
            peaks, _ = nk.ecg_peaks(
                ecg_clean,
                sampling_rate=sampling_frequency,
                correct_artifacts=True
            )

            # 3. Compute FULL HRV feature set
            hrv_full = nk.hrv(
                peaks,
                sampling_rate=sampling_frequency,
                show=False
            )

            # Ensure row is a proper 1-row DataFrame and add ecg_index
            hrv_full = hrv_full.copy()
            hrv_full["ecg_index"] = i

            hrv_features_train.append(hrv_full)

        except Exception as e:
            print(f"Error processing TRAIN ECG {i}: {e}")

            if hrv_features_train:
                empty = pd.DataFrame(
                    [np.nan] * hrv_features_train[0].shape[1],
                    index=hrv_features_train[0].columns
                ).T
                empty["ecg_index"] = i
                hrv_features_train.append(empty)

    # Combine to single DataFrame
    hrv_train = pd.concat(hrv_features_train, ignore_index=True)

    print("hrv_train shape:", hrv_train.shape)
    hrv_train.head()

In [ ]:
if LoadPremadeDataset == False:
    # Merge our new dataframe with our extra variables
    hrv_train = pd.merge(hrv_train, ecg_metrics, on='ecg_index', how='left')
    hrv_train.head()

In [ ]:
if LoadPremadeDataset == False:
    # Remove all columns from the dataframe that contain more than 50% NaN
    threshold = 0.5
    hrv_train_clean = hrv_train.dropna(thresh=len(hrv_train) * threshold, axis=1)

    # Remove all rows that are all NaN
    hrv_train_clean = hrv_train_clean.dropna(how='all')

    hrv_train_clean.to_csv("data/hrv_train.csv", index=False)
    hrv_train_clean.head()

### FULL HRV feature extraction for all ECGs (TEST)

In [ ]:
if LoadPremadeDataset == False:
    hrv_features_test = []

    for i in tqdm(test_idx, desc="HRV (ALL FEATURES): TEST SET"):
        ecg = df.iloc[i].astype(float).values

        try:
            # 1. Clean ECG
            ecg_clean = nk.ecg_clean(ecg, sampling_rate=sampling_frequency)

            # 2. Detect R-peaks
            peaks, _ = nk.ecg_peaks(
                ecg_clean,
                sampling_rate=sampling_frequency,
                correct_artifacts=True
            )

            # 3. Compute FULL HRV feature set
            hrv_full = nk.hrv(
                peaks,
                sampling_rate=sampling_frequency,
                show=False
            )

            # Same as TRAIN: keep as 1-row DataFrame, add index
            hrv_full = hrv_full.copy()
            hrv_full["ecg_index"] = i

            hrv_features_test.append(hrv_full)

        except Exception as e:
            print(f"Error processing TEST ECG {i}: {e}")

            if hrv_features_test:
                empty = pd.DataFrame(
                    [np.nan] * hrv_features_test[0].shape[1],
                    index=hrv_features_test[0].columns
                ).T
                empty["ecg_index"] = i
                hrv_features_test.append(empty)

    hrv_test = pd.concat(hrv_features_test, ignore_index=True)

    print("hrv_test shape:", hrv_test.shape)
    hrv_test.head()

In [ ]:
if LoadPremadeDataset == False:
    # Merge our new dataframe with our extra variables
    hrv_test = pd.merge(hrv_test, ecg_metrics, on='ecg_index', how='left')
    hrv_test.head()

In [ ]:
if LoadPremadeDataset == False:
    # Remove all columns from the dataframe that contain more than 50% NaN
    threshold = 0.5
    hrv_test_clean = hrv_test.dropna(thresh=len(hrv_test) * threshold, axis=1)

    # Remove all rows that are all NaN
    hrv_test_clean = hrv_test_clean.dropna(how='all')

    hrv_test_clean.head()

    hrv_test.to_csv("data/hrv_test.csv", index=False)

# If you don't want to wait that long

In [ ]:
if LoadPremadeDataset == True:
    hrv_test_clean = pd.read_csv("data/hrv_test.csv")
    hrv_train_clean= pd.read_csv("data/hrv_train.csv")

### Feature exploration

In [ ]:
# Merge the HRV data with the rhythm labels
hrv_train_with_labels = pd.merge(
    hrv_train_clean, df_labels, left_on='ecg_index', right_index=True
).reset_index(drop=True)

plt.figure(figsize=(10, 6))

selectedMetric = 'HRV_MedianNN'
rhythms = hrv_train_with_labels['label'].unique()
for rhythm in rhythms:
    subset = hrv_train_with_labels[hrv_train_with_labels['label'] == rhythm]
    plt.hist(subset[selectedMetric], alpha=0.7, label=rhythm, bins='auto')

plt.xlabel(selectedMetric)
plt.ylabel('Frequency')
plt.title('Distribution by Rhythm')
plt.legend()
plt.show()

# Preprocessing

## Missingness

In [ ]:
def check_missing_hrv(df):
    """
    Summarize missingness across HRV features.
    """
    missing = df.isna().sum()
    out = pd.DataFrame({
        "feature": df.columns,
        "missing_n": missing,
        "missing_%": (missing / len(df)) * 100
    })
    display(out.sort_values("missing_%", ascending=False))
    return out

In [ ]:
check_missing_hrv(hrv_train_clean)

## Outlier Detection

In [ ]:
# Function to identify outliers in the data
def identify_outliers(df, column_name, threshold=1.5):
    # Calculate Q1, Q3, and IQR
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1

    # Define outlier bounds
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR

    # Identify outliers
    row_indices = df[(df[column_name] < lower_bound) | (df[column_name] > upper_bound)].index.tolist()
    outlier_values = df.loc[row_indices, column_name].tolist()

    return row_indices, outlier_values, lower_bound, upper_bound

In [ ]:
# Outlier detection ONLY ON TRAIN

# Merge labels with TRAIN features (cleaned hrv_train)
hrv_train_with_labels = pd.merge(
    hrv_train_clean, df_labels, left_on="ecg_index", right_index=True
)

# Outlier detection ONLY on TRAIN
train_outlier_idx, outlier_values, iqr_lower, iqr_upper = identify_outliers(
    hrv_train_with_labels,
    "HRV_MedianNN",
    threshold=1.5
)

# ecg_index as (int)
hrv_train_with_labels["ecg_index"] = hrv_train_with_labels["ecg_index"].astype(int)

print("Train outliers detected:", len(train_outlier_idx))
print("Row indices (in hrv_train_with_labels) with outliers:", train_outlier_idx)
print("Outlier HRV_MedianNN values:", outlier_values)

In [ ]:
# Visualise one outlier ECG

example_outlier_row = train_outlier_idx[0]

# Single row
row = hrv_train_with_labels.loc[example_outlier_row]

# Extract ECG index value
ecg_index_values = row.filter(like="ecg_index").values

# Use first value
ecg_idx = int(ecg_index_values[0])

# Extract raw ECG from df
ecg_raw = df.iloc[ecg_idx].astype(float).values

# Visualise R-Peaks
peaks_outlier, info_outlier = nk.ecg_peaks(
    ecg_raw,
    sampling_rate=sampling_frequency,
    correct_artifacts=True,
    show=True
)

In [ ]:
hrv_train_with_labels.loc[example_outlier_row]

#### **Outliers TEST set** done the same way as for TRAINING

In [ ]:
# Align TEST columns to TRAIN columns

# Align TEST columns to TRAIN columns (no leakage, same feature space)
train_cols = hrv_train_clean.columns  # already cleaned on TRAIN
shared_cols = [c for c in train_cols if c in hrv_test_clean.columns]

hrv_test_aligned = hrv_test_clean[shared_cols].copy()

# Merge TEST HRV with labels
hrv_test_with_labels = pd.merge(
    hrv_test_aligned,
    df_labels[["label", "classification"]],
    left_on="ecg_index",
    right_index=True
)

In [ ]:
# Same IQR bounds as on hrv_train

Q1 = hrv_train_clean["HRV_MedianNN"].quantile(0.25)
Q3 = hrv_train_clean["HRV_MedianNN"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

hrv_test_clean = hrv_test_with_labels[
    (hrv_test_with_labels["HRV_MedianNN"] >= lower_bound) &
    (hrv_test_with_labels["HRV_MedianNN"] <= upper_bound)
    ].copy()

print("hrv_test shape:", hrv_test_clean.shape)
print("hrv_test_with_labels shape:", hrv_test_with_labels.shape)
print("hrv_test_clean shape:", hrv_test_clean.shape)

## Distribution TRAIN + TEST | Sanity check

In [ ]:
print(hrv_train_clean.columns[:5])
print(hrv_test_clean.columns[:5])
print(hrv_test_clean[["HRV_MedianNN", "classification"]].head())

In [ ]:
for feat in ["HRV_MedianNN", "HRV_SDNN"]:
    plt.figure(figsize=(6, 4))
    sns.kdeplot(
        data=hrv_train_clean, x=feat, label="Train", fill=True, common_norm=False
    )
    sns.kdeplot(
        data=hrv_test_clean, x=feat, label="Test", fill=True, common_norm=False, color="orange"
    )
    plt.title(f"{feat}: Train vs Test")
    plt.legend()
    plt.show()

### Outlier Handling TRAIN

#### Winsorising outliers

In [ ]:
Q1 = hrv_train_with_labels["HRV_MedianNN"].quantile(0.25)
Q3 = hrv_train_with_labels["HRV_MedianNN"].quantile(0.75)
IQR = Q3 - Q1

lower_clip = Q1 - 1.5 * IQR
upper_clip = Q3 + 1.5 * IQR

hrv_train_winsor = hrv_train_with_labels.copy()
hrv_train_winsor["HRV_MedianNN_winsor"] = hrv_train_with_labels["HRV_MedianNN"].clip(
    lower=lower_clip, upper=upper_clip
)

print("Shape after winsorizing (same as original):", hrv_train_winsor.shape)

### Outlier Handling Comparison

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(hrv_train_with_labels["HRV_MedianNN"], kde=True, color="red", label="Original")
sns.histplot(hrv_train_winsor["HRV_MedianNN_winsor"], kde=True, color="green", label="Winsorized")

plt.legend()
plt.title("Outlier Handling Comparison")
plt.show()

# Final Preprocessing: Building ML Matrices (X_train, X_test)

In [ ]:
# Select HRV feature columns only
feature_cols = [col for col in hrv_train_with_labels.columns if col.startswith("HRV_")]

# TRAIN data
x_train = hrv_train_with_labels[feature_cols].copy()
y_train = hrv_train_with_labels["classification"].copy()

# TEST data
x_test = hrv_test_clean[feature_cols].copy()
y_test = hrv_test_clean["classification"].copy()

In [ ]:
# Replace +/- inf with NaN in both TRAIN and TEST
for df_ in (x_train, x_test):
    df_.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop columns that are all-NaN (if any)
all_nan_cols = x_train.columns[x_train.isna().all()]
if len(all_nan_cols) > 0:
    print("Dropping all-NaN columns before imputation:", list(all_nan_cols))
    x_train.drop(columns=all_nan_cols, inplace=True)
    x_test.drop(columns=all_nan_cols, inplace=True)

## Imputation

In [ ]:
# Median imputation
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

# X_train: fit_transform
X_train_imputed = imputer.fit_transform(x_train)

# X_test: only transform (so test set remains untouched)
X_test_imputed = imputer.transform(x_test)

## Normalisation

In [ ]:
#Temporarily convert to DataFrame to calculate Skewness easily
temp_df = pd.DataFrame(X_train_imputed, columns=feature_cols)
skewness = temp_df.skew().sort_values(ascending=False)

#Identify skewed columns (Threshold > 1.0)
skewed_cols = skewness[abs(skewness) > 1.0].index.tolist()

#Apply Log Transform directly to the NumPy arrays
for col_name in skewed_cols:
    # Find the column index (integer position)
    col_idx = feature_cols.index(col_name)

    # Check for negative values (Log crashes on negatives)
    # We find the global minimum for this column across Train and Test
    min_val = min(X_train_imputed[:, col_idx].min(), X_test_imputed[:, col_idx].min())

    shift = 0
    if min_val < 0:
        # If negatives exist, calculate a shift to make the minimum 0
        shift = abs(min_val)

    # Apply transformation in-place: Log(x + shift + 1)
    X_train_imputed[:, col_idx] = np.log1p(X_train_imputed[:, col_idx] + shift)
    X_test_imputed[:, col_idx] = np.log1p(X_test_imputed[:, col_idx] + shift)

## Scaling

In [ ]:
scaler = RobustScaler()

# X_train: fit_transform
X_train_scaled = scaler.fit_transform(X_train_imputed)

# X_test: only transform (so test set remains untouched)
X_test_scaled = scaler.transform(X_test_imputed)

# Convert back to df with column names
x_train = pd.DataFrame(X_train_scaled, columns=feature_cols)
x_test = pd.DataFrame(X_test_scaled, columns=feature_cols)

### **Sanity checks**

In [ ]:
# Median X_train
print("Median of scaled features (should be ~0):")
print(x_train.median().round(3))

# IQR X_train
print("\nIQR of scaled features (should be ~1):")
print((x_train.quantile(0.75) - x_train.quantile(0.25)).round(3))

#Checking skewness of the datasets
skewness_train = x_train.skew().sort_values(ascending=False)
skewness_test = x_train.skew().sort_values(ascending=False)
# Filter for highly skewed columns (absolute skew > 1.0)
high_skew_cols_train = skewness_train[abs(skewness_train) > 1.0]
high_skew_cols_test = skewness_test[abs(skewness_test) > 1.0]

print(len(high_skew_cols_train))
print(len(high_skew_cols_test))

# Final ML datasets (X_train, X_test, y_train, y_test

In [ ]:
print("Train size:", len(train_idx))
print("Test size:", len(test_idx))

In [ ]:
if True:
    # Feature matrices (winsorised > imputation > scaling)
    x_train = X_train_scaled
    x_test = X_test_scaled

    # Target vectors (created earlier from HRV + labels AF(0/1))
    y_train = hrv_train_with_labels["classification"].copy()
    y_test = hrv_test_clean["classification"].copy()

    print("Final X_train shape:", x_train.shape)
    print("Final X_test shape:", x_test.shape)
    print("Final y_train shape:", y_train.shape)
    print("Final y_test shape:", y_test.shape)

# Machine Learning Training Setup

## Safety check

In [ ]:
assert len(x_train) == len(y_train), "Misaligned TRAIN matrix and labels!"
assert len(x_test) == len(y_test), "Misaligned TEST matrix and labels!"

assert not np.isnan(x_train).any(), "NaNs detected in X_train!"
assert not np.isnan(x_test).any(), "NaNs detected in X_test!"

## Comparison framework

In [ ]:
resultsTable = pd.DataFrame(columns=['Model', 'Accuracy', 'F1 Score', 'Precision', 'Recall', 'ROC', 'ROC_AUC', 'cm'])

def modelResults(model, accuracy, f1, precision, recall, roc_auc, roc_cur, cm):
    print(
        f"Model {model} evaluated. \nAccuracy: {accuracy} \nF1 Score: {f1} \nPrecision: {precision} \nRecall: {recall} \nROC AUC: {roc_auc}")
    resultsTable.loc[len(resultsTable)] = [model, accuracy, f1, precision, recall, roc_cur, roc_auc, cm]
    resultsTable.to_csv("data/trainingResults.csv", index=False, mode="a")

In [ ]:
hyperParameterResultsTable = pd.DataFrame(columns=['Model', 'Accuracy', 'F1 Score', 'Precision', 'Recall', 'ROC', 'ROC_AUC', 'cm'])

def hyperParameterResults(model, accuracy, f1, precision, recall, roc_auc, roc_cur, cm):
    print(
        f"Model {model} evaluated. \nAccuracy: {accuracy} \nF1 Score: {f1} \nPrecision: {precision} \nRecall: {recall} \nROC AUC: {roc_auc}")
    hyperParameterResultsTable.loc[len(resultsTable)] = [model, accuracy, f1, precision, recall, roc_cur, roc_auc, cm]
    hyperParameterResultsTable.to_csv("data/hyperParameterResults.csv", index=False, mode="a")


In [ ]:
print(
    f"X train length: {len(x_train)}\n X test length: {len(x_test)} \n Y train length: {len(y_train)}\n Y test length: {len(y_test)}")

# Machine Learning Training

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler

#Going back to basics, the currently used x_train and x_test gave ValueErrors as negative values for Log

raw_cols = [c for c in hrv_train_with_labels.columns if c.startswith("HRV_")]
raw_train = hrv_train_with_labels[raw_cols].copy()
raw_test = hrv_test_clean[raw_cols].copy()

raw_train.replace([np.inf, -np.inf], np.nan, inplace=True)
raw_test.replace([np.inf, -np.inf], np.nan, inplace=True)

imputer_eng = SimpleImputer(strategy="median")
raw_train_imp = pd.DataFrame(imputer_eng.fit_transform(raw_train), columns=raw_cols)
raw_test_imp = pd.DataFrame(imputer_eng.transform(raw_test), columns=raw_cols)

skewness = raw_train_imp.skew().sort_values(ascending=False)
skewed_cols = skewness[abs(skewness) > 1.0].index.tolist()

new_features_train = pd.DataFrame(index=raw_train_imp.index)
new_features_test = pd.DataFrame(index=raw_test_imp.index)

#1. Log Transforms
for col in skewed_cols:
    # +1e-6 avoids log(0)
    new_features_train[f'Log_{col}'] = np.log(raw_train_imp[col] + 1e-6)
    new_features_test[f'Log_{col}'] = np.log(raw_test_imp[col] + 1e-6)

#2. 2. Coefficient of Variation (CV) computation:
if 'HRV_SDNN' in raw_train_imp.columns and 'HRV_MeanNN' in raw_train_imp.columns:
    new_features_train['CV_SDNN'] = raw_train_imp['HRV_SDNN'] / (raw_train_imp['HRV_MeanNN'] + 1e-6)
    new_features_test['CV_SDNN'] = raw_test_imp['HRV_SDNN'] / (raw_test_imp['HRV_MeanNN'] + 1e-6)

# 3. Chaos Index (Amplifies the "irregularly irregular" signal specific to AF.):
entropy_col = 'HRV_ApEn' if 'HRV_ApEn' in raw_train_imp.columns else 'HRV_SampEn'
if 'HRV_RMSSD' in raw_train_imp.columns and entropy_col in raw_train_imp.columns:
    new_features_train['Chaos_Index'] = raw_train_imp['HRV_RMSSD'] * raw_train_imp[entropy_col]
    new_features_test['Chaos_Index'] = raw_test_imp['HRV_RMSSD'] * raw_test_imp[entropy_col]

new_features_train.replace([np.inf, -np.inf], np.nan, inplace=True)
new_features_test.replace([np.inf, -np.inf], np.nan, inplace=True)

imputer_new = SimpleImputer(strategy="median")
new_train_clean = pd.DataFrame(imputer_new.fit_transform(new_features_train), columns=new_features_train.columns)
new_test_clean = pd.DataFrame(imputer_new.transform(new_features_test), columns=new_features_test.columns)

scaler_eng = RobustScaler()
new_train_scaled = pd.DataFrame(scaler_eng.fit_transform(new_train_clean), columns=new_features_train.columns)
new_test_scaled = pd.DataFrame(scaler_eng.transform(new_test_clean), columns=new_features_test.columns)

if not isinstance(x_train, pd.DataFrame):
    x_train = pd.DataFrame(x_train, columns=feature_cols)
if not isinstance(x_test, pd.DataFrame):
    x_test = pd.DataFrame(x_test, columns=feature_cols)

x_train_added = pd.concat([x_train, new_train_scaled], axis=1)
x_test_added = pd.concat([x_test, new_test_scaled], axis=1)

## Soft voting classifier

### Hyperparameter sweep

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, roc_curve, \
    confusion_matrix
import numpy as np
import pandas as pd

X_tr = x_train_added if 'x_train_added' in globals() else x_train
X_te = x_test_added if 'x_test_added' in globals() else x_test

if isinstance(X_tr, np.ndarray):
    X_tr_use = X_tr
    X_te_use = X_te
else:
    X_tr_use = X_tr.values
    X_te_use = X_te.values

y_tr_use = y_train.values if isinstance(y_train, pd.Series) else y_train
y_te_use = y_test.values if isinstance(y_test, pd.Series) else y_test

solvers_grid = ['lbfgs', 'liblinear', 'saga']
penalties_by_solver = {
    'lbfgs': ['l2', None],
    'liblinear': ['l1', 'l2'],
    'saga': ['l1', 'l2', 'elasticnet']
}
Cs = [0.01, 0.1, 1.0, 10.0, 100.0]
class_weights = [None, 'balanced']
max_iter = 2000
l1_ratio_values = [0.0, 0.25, 0.5, 0.75]

for solver in solvers_grid:
    for penalty in penalties_by_solver[solver]:
        for C in Cs:
            for cw in class_weights:
                if penalty == 'elasticnet' and solver == 'saga':
                    for l1r in l1_ratio_values:
                        try:
                            model = LogisticRegression(
                                solver=solver,
                                penalty=penalty,
                                C=C,
                                l1_ratio=l1r,
                                class_weight=cw,
                                max_iter=max_iter,
                                n_jobs=-1 if solver in ['lbfgs', 'saga'] else None,
                                random_state=3003
                            )
                            model.fit(X_tr_use, y_tr_use)
                            y_pred = model.predict(X_te_use)
                            y_proba = model.predict_proba(X_te_use)[:, 1]
                            acc = accuracy_score(y_te_use, y_pred)
                            f1 = f1_score(y_te_use, y_pred)
                            prec = precision_score(y_te_use, y_pred)
                            rec = recall_score(y_te_use, y_pred)
                            auc = roc_auc_score(y_te_use, y_proba)
                            roc_cur = roc_curve(y_te_use, y_proba)
                            cm = confusion_matrix(y_true=y_te_use, y_pred=y_pred, normalize='true')
                            name = f"LR(solver={solver},penalty={penalty},C={C},class_weight={cw},l1_ratio={l1r})"
                            hyperParameterResults(name, acc, f1, prec, rec, auc, roc_cur, cm)
                        except Exception as e:
                            continue
                else:
                    try:
                        kwargs = dict(
                            solver=solver,
                            penalty=penalty if penalty is not None else 'l2',
                            C=C,
                            class_weight=cw,
                            max_iter=max_iter,
                            n_jobs=-1 if solver in ['lbfgs', 'saga'] else None,
                            random_state=3003
                        )
                        if penalty != 'elasticnet' and 'l1_ratio' in kwargs:
                            kwargs.pop('l1_ratio', None)
                        if solver == 'liblinear' and kwargs.get('n_jobs') is not None:
                            kwargs.pop('n_jobs', None)

                        model = LogisticRegression(**kwargs)
                        model.fit(X_tr_use, y_tr_use)
                        y_pred = model.predict(X_te_use)
                        y_proba = model.predict_proba(X_te_use)[:, 1]
                        acc = accuracy_score(y_te_use, y_pred)
                        f1 = f1_score(y_te_use, y_pred)
                        prec = precision_score(y_te_use, y_pred)
                        rec = recall_score(y_te_use, y_pred)
                        auc = roc_auc_score(y_te_use, y_proba)
                        roc_cur = roc_curve(y_te_use, y_proba)
                        cm = confusion_matrix(y_true=y_te_use, y_pred=y_pred, normalize='true')
                        name = f"LR(solver={solver},penalty={penalty},C={C},class_weight={cw})"
                        hyperParameterResults(name, acc, f1, prec, rec, auc, roc_cur, cm)
                    except Exception as e:
                        continue


In [70]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, roc_curve, \
    confusion_matrix
import numpy as np
import pandas as pd

X_tr = x_train_added if 'x_train_added' in globals() else x_train
X_te = x_test_added if 'x_test_added' in globals() else x_test

if isinstance(X_tr, np.ndarray):
    X_tr_use = X_tr
    X_te_use = X_te
else:
    X_tr_use = X_tr.values
    X_te_use = X_te.values

y_tr_use = y_train.values if isinstance(y_train, pd.Series) else y_train
y_te_use = y_test.values if isinstance(y_test, pd.Series) else y_test

n_estimators_grid = [50, 100, 200, 300, 500]
max_depth_grid = [None, 5, 10, 20, 30]
min_samples_split_grid = [2, 5, 10]
min_samples_leaf_grid = [1, 2, 4]
max_features_grid = ['sqrt', 'log2', None]
bootstrap_grid = [True, False]
class_weight_grid = [None, 'balanced', 'balanced_subsample']
criterion_grid = ['gini', 'entropy', 'log_loss']
oob_score_grid = [False, True]

for n_est in n_estimators_grid:
    for crit in criterion_grid:
        for md in max_depth_grid:
            for mss in min_samples_split_grid:
                for msl in min_samples_leaf_grid:
                    for mf in max_features_grid:
                        for bs in bootstrap_grid:
                            if (not bs):
                                oob = False
                                oob_list = [False]
                            else:
                                oob_list = oob_score_grid
                            for oob in oob_list:
                                for cw in class_weight_grid:
                                    try:
                                        model = RandomForestClassifier(
                                            n_estimators=n_est,
                                            criterion=crit,
                                            max_depth=md,
                                            min_samples_split=mss,
                                            min_samples_leaf=msl,
                                            max_features=mf,
                                            bootstrap=bs,
                                            oob_score=oob,
                                            class_weight=cw,
                                            n_jobs=-1,
                                            random_state=3003
                                        )
                                        model.fit(X_tr_use, y_tr_use)
                                        y_pred = model.predict(X_te_use)
                                        y_proba = model.predict_proba(X_te_use)[:, 1]
                                        acc = accuracy_score(y_te_use, y_pred)
                                        f1 = f1_score(y_te_use, y_pred)
                                        prec = precision_score(y_te_use, y_pred)
                                        rec = recall_score(y_te_use, y_pred)
                                        auc = roc_auc_score(y_te_use, y_proba)
                                        roc_cur = roc_curve(y_te_use, y_proba)
                                        cm = confusion_matrix(y_true=y_te_use, y_pred=y_pred, normalize='true')
                                        name = f"RF(n_estimators={n_est},criterion={crit},max_depth={md},min_samples_split={mss},min_samples_leaf={msl},max_features={mf},bootstrap={bs},oob_score={oob},class_weight={cw})"
                                        hyperParameterResults(name, acc, f1, prec, rec, auc, roc_cur, cm)
                                    except Exception:
                                        continue


Model RF(n_estimators=300,criterion=entropy,max_depth=None,min_samples_split=5,min_samples_leaf=4,max_features=None,bootstrap=False,oob_score=False,class_weight=None) evaluated. 
Accuracy: 0.9460067491563554 
F1 Score: 0.75 
Precision: 0.8089887640449438 
Recall: 0.6990291262135923 
ROC AUC: 0.8691605523851873
Model RF(n_estimators=300,criterion=entropy,max_depth=None,min_samples_split=5,min_samples_leaf=4,max_features=None,bootstrap=False,oob_score=False,class_weight=balanced) evaluated. 
Accuracy: 0.9448818897637795 
F1 Score: 0.776255707762557 
Precision: 0.7327586206896551 
Recall: 0.8252427184466019 
ROC AUC: 0.904197238074063
Model RF(n_estimators=300,criterion=entropy,max_depth=None,min_samples_split=5,min_samples_leaf=4,max_features=None,bootstrap=False,oob_score=False,class_weight=balanced_subsample) evaluated. 
Accuracy: 0.9448818897637795 
F1 Score: 0.776255707762557 
Precision: 0.7327586206896551 
Recall: 0.8252427184466019 
ROC AUC: 0.904197238074063
Model RF(n_estimators=

KeyboardInterrupt: 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, roc_curve, \
    confusion_matrix
import numpy as np
import pandas as pd

X_tr = x_train_added if 'x_train_added' in globals() else x_train
X_te = x_test_added if 'x_test_added' in globals() else x_test

if isinstance(X_tr, np.ndarray):
    X_tr_use = X_tr
    X_te_use = X_te
else:
    X_tr_use = X_tr.values
    X_te_use = X_te.values

y_tr_use = y_train.values if isinstance(y_train, pd.Series) else y_train
y_te_use = y_test.values if isinstance(y_test, pd.Series) else y_test

n_neighbors_grid = list(range(1, 51))
weights_grid = ['uniform', 'distance']
metric_grid = ['minkowski', 'euclidean', 'manhattan', 'chebyshev']
p_by_metric = {'minkowski': [1, 2, 3], 'euclidean': [2], 'manhattan': [1], 'chebyshev': [None]}
leaf_size_grid = [15, 30, 45, 60]
algorithm_grid = ['auto', 'ball_tree', 'kd_tree', 'brute']

for n in n_neighbors_grid:
    for w in weights_grid:
        for alg in algorithm_grid:
            for leaf in leaf_size_grid:
                for m in metric_grid:
                    for p in p_by_metric[m]:
                        try:
                            kwargs = dict(
                                n_neighbors=n,
                                weights=w,
                                algorithm=alg,
                                leaf_size=leaf,
                                metric=m,
                                n_jobs=-1
                            )
                            if p is not None and m == 'minkowski':
                                kwargs['p'] = p
                            elif m != 'minkowski' and 'p' in kwargs:
                                kwargs.pop('p', None)
                            elif m == 'minkowski' and p is None:
                                kwargs['p'] = 2

                            model = KNeighborsClassifier(**kwargs)
                            model.fit(X_tr_use, y_tr_use)
                            y_pred = model.predict(X_te_use)
                            if hasattr(model, "predict_proba"):
                                y_proba = model.predict_proba(X_te_use)[:, 1]
                            else:
                                # Fallback using distance to neighbors if proba not available
                                # For KNN classifiers with weights this should exist; safeguard:
                                y_proba = y_pred.astype(float)

                            acc = accuracy_score(y_te_use, y_pred)
                            f1 = f1_score(y_te_use, y_pred)
                            prec = precision_score(y_te_use, y_pred)
                            rec = recall_score(y_te_use, y_pred)

                            try:
                                auc = roc_auc_score(y_te_use, y_proba)
                                roc_cur = roc_curve(y_te_use, y_proba)
                            except Exception:
                                # In rare cases when y_proba is degenerate
                                auc = float('nan')
                                roc_cur = (np.array([0.0, 1.0]), np.array([0.0, 1.0]), np.array([0.5]))

                            cm = confusion_matrix(y_true=y_te_use, y_pred=y_pred, normalize='true')
                            name = f"KNN(n_neighbors={n},weights={w},algorithm={alg},leaf_size={leaf},metric={m}" + (
                                f",p={p}" if m == 'minkowski' else "") + ")"
                            hyperParameterResults(name, acc, f1, prec, rec, auc, roc_cur, cm)
                        except Exception:
                            continue


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, roc_curve, \
    confusion_matrix
import numpy as np
import pandas as pd

X_tr = x_train_added if 'x_train_added' in globals() else x_train
X_te = x_test_added if 'x_test_added' in globals() else x_test

if isinstance(X_tr, np.ndarray):
    X_tr_use = X_tr
    X_te_use = X_te
else:
    X_tr_use = X_tr.values
    X_te_use = X_te.values

y_tr_use = y_train.values if isinstance(y_train, pd.Series) else y_train
y_te_use = y_test.values if isinstance(y_test, pd.Series) else y_test

n_estimators_grid = [50, 100, 200, 300, 500, 800, 1000, 2000, 4000, 5000]
learning_rate_grid = [0.01, 0.05, 0.1, 0.2, 0.5, 1.0]
algorithm_grid = ['SAMME', 'SAMME.R']

dt_max_depth_grid = [1, 2, 3, 4, 5]
dt_min_samples_split_grid = [2, 5, 10]
dt_min_samples_leaf_grid = [1, 2, 4]

for n_est in n_estimators_grid:
    for lr in learning_rate_grid:
        for algo in algorithm_grid:
            for md in dt_max_depth_grid:
                for mss in dt_min_samples_split_grid:
                    for msl in dt_min_samples_leaf_grid:
                        try:
                            base = DecisionTreeClassifier(
                                max_depth=md,
                                min_samples_split=mss,
                                min_samples_leaf=msl,
                                random_state=3003
                            )
                            model = AdaBoostClassifier(
                                estimator=base,
                                n_estimators=n_est,
                                learning_rate=lr,
                                algorithm=algo,
                                random_state=3003
                            )
                            model.fit(X_tr_use, y_tr_use)
                            y_pred = model.predict(X_te_use)
                            if hasattr(model, "predict_proba"):
                                y_proba = model.predict_proba(X_te_use)[:, 1]
                            else:
                                if hasattr(model, "decision_function"):
                                    scores = model.decision_function(X_te_use)
                                    y_proba = (scores - scores.min()) / (scores.max() - scores.min() + 1e-12)
                                else:
                                    y_proba = y_pred.astype(float)

                            acc = accuracy_score(y_te_use, y_pred)
                            f1 = f1_score(y_te_use, y_pred)
                            prec = precision_score(y_te_use, y_pred)
                            rec = recall_score(y_te_use, y_pred)

                            try:
                                auc = roc_auc_score(y_te_use, y_proba)
                                roc_cur = roc_curve(y_te_use, y_proba)
                            except Exception:
                                auc = float('nan')
                                roc_cur = (np.array([0.0, 1.0]), np.array([0.0, 1.0]), np.array([0.5]))

                            cm = confusion_matrix(y_true=y_te_use, y_pred=y_pred, normalize='true')
                            name = f"AdaBoost(n_estimators={n_est},lr={lr},alg={algo},DT(max_depth={md},min_split={mss},min_leaf={msl}))"
                            hyperParameterResults(name, acc, f1, prec, rec, auc, roc_cur, cm)
                        except Exception:
                            continue


## Best Voting Classifier Search

In [ ]:
import itertools

# Define the pool of base classifiers with various hyperparameters
classifiers_pool = [
    ('LR_balanced', LogisticRegression(multi_class='auto', max_iter=1000, class_weight='balanced', random_state=3003)),
    ('LR_default', LogisticRegression(multi_class='auto', max_iter=1000, random_state=3003)),
    ('RF_100', RandomForestClassifier(n_estimators=100, random_state=3003)),
    ('RF_200', RandomForestClassifier(n_estimators=200, random_state=3003)),
    ('RF_300', RandomForestClassifier(n_estimators=200, random_state=3003)),
    ('KNN_5', KNeighborsClassifier(n_neighbors=5)),
    ('KNN_7', KNeighborsClassifier(n_neighbors=7)),
    ('GBC_100', GradientBoostingClassifier(n_estimators=100, random_state=3003)),
    ('GBC_200', GradientBoostingClassifier(n_estimators=200, random_state=3003)),
    ('Ada_50', AdaBoostClassifier(n_estimators=50, random_state=3003)),
    ('Ada_100', AdaBoostClassifier(n_estimators=100, random_state=3003))
]

best_f1_found = -1
best_voting_model = None
best_ensemble_name = ""

print("Searching for best Voting Classifier configuration (optimizing for F1 Score)...")

# Iterate through all possible combinations of length 2 to 4 (limiting to 4 to avoid too long runtime)
for r in range(2, 5):
    for ensemble in itertools.combinations(classifiers_pool, r):
        # Create a name for this combination
        names = [name for name, _ in ensemble]
        ensemble_name = f"BestVote ({'+'.join(names)})"

        # Create the voting classifier
        # Using soft voting as these models support probability estimates
        voter = VotingClassifier(estimators=list(ensemble), voting='soft', n_jobs=-1)

        # Train
        voter.fit(x_train_added, y_train)

        # Predict
        y_pred = voter.predict(x_test_added)
        y_pred_proba = voter.predict_proba(x_test_added)[:, 1]

        # Evaluate
        current_f1 = f1_score(y_test, y_pred)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        roc_cur = roc_curve(y_test, y_pred_proba)
        cm = confusion_matrix(y_true=y_test, y_pred=y_pred, normalize='true')
        modelResults(ensemble, accuracy, current_f1, precision, recall, roc_auc, roc_cur, cm)



        # print(f"Tested {ensemble_name}: F1 Score = {current_f1:.4f}")

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        roc_cur = roc_curve(y_test, y_pred_proba)
        cm = confusion_matrix(y_true=y_test, y_pred=y_pred, normalize='true')

        modelResults(ensemble_name, accuracy, current_f1, precision, recall, roc_auc, roc_cur, cm)

        if current_f1 > best_f1_found:
            best_f1_found = current_f1
            best_voting_model = voter
            best_ensemble_name = ensemble_name

print(f"\nWinner configuration: {best_ensemble_name} with F1 Score: {best_f1_found:.4f}")

# Log the best result to the global resultsTable
if best_voting_model:
    y_pred = best_voting_model.predict(x_test)
    y_pred_proba = best_voting_model.predict_proba(x_test)[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    roc_cur = roc_curve(y_test, y_pred_proba)
    cm = confusion_matrix(y_true=y_test, y_pred=y_pred, normalize='true')

    # Plot matrix for the winner
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_voting_model.classes_)
    disp.plot()
    plt.title(f"Best Found Ensemble: {best_ensemble_name}")
    plt.show()

In [71]:
hyperParameterResultsTable.to_csv("data/hyperParameterResults.csv", index=False, mode="a")

# Model evaluation
## Quick conclusion

In [ ]:
original_size = resultsTable.shape[0]
resultsTable = resultsTable.drop_duplicates(subset=['Model', 'Accuracy', 'F1 Score', 'Precision', 'Recall', 'ROC_AUC'])
print(f"Dropped {original_size - resultsTable.shape[0]} duplicate rows")

In [ ]:
resultsTable

In [72]:
top_models = hyperParameterResultsTable.sort_values(by='F1 Score', ascending=False).head(5)

print("Top 5 models based on F1 score:")
for i, (_, row) in enumerate(top_models.iterrows(), 1):
    print(f"{i}. {row['Model']} with an F1 score of {row['F1 Score']:.4f}")


Top 5 models based on F1 score:
1. LR(solver=lbfgs,penalty=l2,C=0.01,class_weight=None) with an F1 score of 0.7629


## Graphs of numerical metrics
### Logarithmic scale

In [ ]:
if True:
    numeric_metrics = ['Accuracy', 'F1 Score', 'Precision', 'Recall', 'ROC_AUC']

    model_labels = [str(model).split('(')[0] for model in resultsTable['Model']]

    fig, axes = plt.subplots(len(numeric_metrics), 1, figsize=(10, len(numeric_metrics) * 8))

    for i, col in enumerate(numeric_metrics):
        bars = axes[i].bar(range(len(resultsTable)), resultsTable[col])
        axes[i].set_xticks(range(len(resultsTable)))
        axes[i].set_xticklabels(model_labels, rotation=45, ha='right')
        axes[i].set_ylabel(col)
        axes[i].set_title(f'{col} by Model')
        axes[i].set_yscale('log')
        axes[i].grid(axis='y', alpha=0.3)

        for j, bar in enumerate(bars):
            height = bar.get_height()
            axes[i].text(bar.get_x() + bar.get_width() / 2., height,
                         f'{resultsTable[col].iloc[j]:.3f}',
                         ha='center', va='bottom', fontsize=9)

    plt.tight_layout()
    plt.show()

In [ ]:
numeric_metrics = ['Accuracy', 'F1 Score', 'Precision', 'Recall', 'ROC_AUC']

model_labels = [str(model).split('(')[0] for model in resultsTable['Model']]

fig, axes = plt.subplots(len(numeric_metrics), 1, figsize=(10, len(numeric_metrics) * 8))

for i, col in enumerate(numeric_metrics):
    bars = axes[i].bar(range(len(resultsTable)), resultsTable[col])
    axes[i].set_xticks(range(len(resultsTable)))
    axes[i].set_xticklabels(model_labels, rotation=45, ha='right')
    axes[i].set_ylabel(col)
    axes[i].set_title(f'{col} by Model')
    axes[i].set_ylim(top=1)
    axes[i].grid(axis='y', alpha=0.3)

    for j, bar in enumerate(bars):
        height = bar.get_height()
        axes[i].text(bar.get_x() + bar.get_width() / 2., height,
                     f'{resultsTable[col].iloc[j]:.3f}',
                     ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## ROC Curves

In [ ]:
plt.figure(figsize=(20, 20))

for idx, row in resultsTable.iterrows():
    model_name = str(row['Model']).split('(')[0]
    fpr, tpr, thresholds = row['ROC']
    roc_auc = row['ROC_AUC']
    plt.plot(fpr, tpr, lw=2, label=f'{model_name} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier (AUC = 0.500)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves - Model Comparison', fontsize=14)
plt.legend(loc="lower right", fontsize=12)
plt.grid(alpha=1)
plt.tight_layout()
plt.show()